
   # Extraction from IMDB


### Install the following library if not already installed

#### Import Library necessary for the Project

In [1]:
# importing os ans sys modules
import os
import sys

# Navigate to project directory to import modules

sys.path.insert(0, os.path.dirname(os.getcwd()))

# Importing function create from Manga_function.py
from Functions.Manga_function import write_to_file, create_soup, extract_date, extract_episod, extract_link, extract_title 

# MySQL connector
import mysql.connector

# Pandas
import pandas as pd

# importing os module for environment variables
import os

# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv() 


True

#### Connect to the database, create manga table if it doesn't exist and extract the number of row

In [2]:
# Initialize variables
host = os.getenv("MYSQL_HOSTNAME")
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
database = "manga"
table_name = "DemonSlayer"
manga_name = "Demon Slayer"
csv_file = table_name + ".csv"
log_file = table_name + ".txt"
url = os.getenv("URL")

In [3]:
write_to_file(log_file, f"Connecting to MySQL {database} database")

# Connect to the database
try:

    cnx = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=3306
    )

    write_to_file(log_file, f"Connected to {database} database")

    # Create a cursor object
    cursor = cnx.cursor()

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [4]:
write_to_file(log_file, f"Creating table {table_name} into {database} if doesn't exist")

# Create Table into MYSQL
try:

    # Create a cursor object
    cursor = cnx.cursor()

    # SQL statement to create the table
    sql = '''CREATE TABLE IF NOT EXISTS ''' + table_name + ''' (
        Episod INT UNIQUE NOT NULL PRIMARY KEY,
        Season INT NOT NULL,
        Episod_season INT NOT NULL,
        Title VARCHAR(255),
        Day INT,
        Month INT,
        Year INT NOT NULL,
        Rate DOUBLE,
        Link VARCHAR(255)
    )
'''

    # Execute the SQL statement
    cursor.execute(sql)

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file, f"Table {table_name} created into {database} successfully")

except mysql.connector.Error as err:
    print(f"Error: {err}")

In [5]:
write_to_file(log_file, f"Extracting number total of episode and number of episod already in {table_name} table")

# Extract number of One piece episod

number_tot_ep = 63

write_to_file(log_file, f"There are {number_tot_ep} {manga_name} episods")

# Extract number of episod in database
try:
         # Execute a query
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    number_ep_database = cursor.fetchall()
    number_ep_database = int(number_ep_database[0][0])

    write_to_file(log_file, f"There is(are) {number_ep_database} episod(s) already in {table_name} table")


except mysql.connector.Error as err:
    print(f"Error: {err}")

number_ep_to_extract = number_tot_ep - number_ep_database

write_to_file(log_file, f"There is(are) {number_ep_to_extract} episod(s) to extract")

if number_ep_to_extract == 0 :
    write_to_file(log_file, f"Program stopped because there is no data to extract")
    raise SystemExit("System stop because there is no episod to extract")

In [6]:
# find the first link to extract

if number_ep_database == 0:
    link  = url
else:
    # Extract last link in database
    try:
            # Execute a query
        cursor.execute(f"SELECT link from {table_name} order by Episod desc limit 1")
        link = cursor.fetchall()
        link = str(link[0][0])
    except mysql.connector.Error as err:
        print(f"Error: {err}")
           
    soup = create_soup(link)
    link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']
    link = extract_link(link)

In [12]:
# Function to create list if doesn't exist
def create_list(*args):
    """Create list if it doesn't exist
    """
    mydict = {}
    for arg in args:
        if arg not in globals():           
            mydict[arg] = list()
    globals().update(mydict)



create_list("season_list","episod_season_list","episod_list","title_list","rating_list","day_list","month_list","year_list","link_list")

link_list.append(link)

for i in range(number_ep_to_extract):

    try:

        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link}")
        soup = create_soup(link)
        
        # Season and Episod
        ep = soup.find('div', class_="sc-3f4e3993-0 fYpskP").text
        season,episod_season = extract_episod(ep)
        season_list.append(season)
        episod_season_list.append(episod_season)

        # Title
        title = extract_title(soup.find('span', class_="hero__primary-text").text)
        title_list.append(title)

        # Date
        date = soup.find_all('li', class_="ipc-inline-list__item")
        day,month,year = extract_date(date)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)

        #Rating
        rate = soup.find('span', class_="sc-c4ffe080-1 iQZtLP").text.replace(",",".")
        rating_list.append(float(rate))

        write_to_file(log_file, f"Extracted correctly the following episod: Episode: {episod} | Title: {title} | Rate: {rate}")

        #Link 

        if i == 0 and number_ep_database == 0: 
            link = soup.find('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase').attrs['href']
        else:
            link = soup.find_all('a', class_='ipc-icon-button sc-3f4e3993-3 iasCTO ipc-icon-button--baseAlt ipc-icon-button--onBase')[1].attrs['href']

        if i < number_ep_to_extract - 1:        
            link = extract_link(link)
            link_list.append(link)

        i = i+1   

    except IndexError:
        pass
        
    except WebDriverException: 
        write_to_file(log_file, f"Extracting episod {number_ep_database+1+i} from {link} failed")

number_ep_extracted = i 

episode_list = list(range(number_ep_database + 1, number_ep_database + 1 + number_ep_extracted))



NameError: name 'WebDriverException' is not defined

In [15]:
if len(link_list) > len(episod_list):
    del link_list[-1]

In [ ]:
data = {"Episod":episod_list,"Season":season_list,"Episod_season_list":episod_season_list, "Title":title_list, "Day":day_list, "Month":month_list, "Year":year_list, "Rate":rating_list, "link":link_list}
df = pd.DataFrame(data)
df.to_csv(csv_file,index=False,mode= "a")
df

In [17]:
if df.shape[0] == number_ep_to_extract:
    write_to_file(log_file,f"Extracting {number_ep_to_extract} episod successfuly")
else:
    write_to_file(log_file,f"Extraction failed")

In [ ]:
write_to_file(log_file,f"Sending {len(df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(df)} rows to MySQL table '{table_name}'.")
except mysql.connector.Error as e:
    print(f"Error sending data to MySQL: {e}")
finally:
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()